In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [2]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [3]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2016/01/01) 00-01,40.1,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
...,...,...,...
24,(2021/06/01) 23-24,41.18,124
25,Min,41.18,0
26,Max,299.69,124
27,Average,128.6475,67.916666666666666666666666667


In [4]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2016/01/01) 00-01,40.10,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
5,(2016/01/01) 04-05,40.07,502.9
...,...,...,...
20,(2021/06/01) 19-20,299.34,33.0
21,(2021/06/01) 20-21,124.05,60.0
22,(2021/06/01) 21-22,80.14,82.0
23,(2021/06/01) 22-23,48.66,116.0


In [5]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2016/01/01) 00-01,40.10,477.5,2016-01-01,0
2,(2016/01/01) 01-02,40.08,486.6,2016-01-01,1
3,(2016/01/01) 02-03,40.06,495.6,2016-01-01,2
4,(2016/01/01) 03-04,40.08,496.6,2016-01-01,3
5,(2016/01/01) 04-05,40.07,502.9,2016-01-01,4
...,...,...,...,...,...
20,(2021/06/01) 19-20,299.34,33.0,2021-06-01,19
21,(2021/06/01) 20-21,124.05,60.0,2021-06-01,20
22,(2021/06/01) 21-22,80.14,82.0,2021-06-01,21
23,(2021/06/01) 22-23,48.66,116.0,2021-06-01,22


In [6]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
1,40.10,477.5,2016-01-01,0
2,40.08,486.6,2016-01-01,1
3,40.06,495.6,2016-01-01,2
4,40.08,496.6,2016-01-01,3
5,40.07,502.9,2016-01-01,4
...,...,...,...,...
20,299.34,33.0,2021-06-01,19
21,124.05,60.0,2021-06-01,20
22,80.14,82.0,2021-06-01,21
23,48.66,116.0,2021-06-01,22


In [7]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4
2016-01-01,40.08,486.6,1,2016,1,1,4
2016-01-01,40.06,495.6,2,2016,1,1,4
2016-01-01,40.08,496.6,3,2016,1,1,4
2016-01-01,40.07,502.9,4,2016,1,1,4
...,...,...,...,...,...,...,...
2021-06-01,299.34,33.0,19,2021,6,1,1
2021-06-01,124.05,60.0,20,2021,6,1,1
2021-06-01,80.14,82.0,21,2021,6,1,1


In [8]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume  day_of_week       value  price_weight_av  price_max  \
year month day                                                                
2016 1     1    7873.8           96  336882.508        42.785251      60.08   
           2    5396.5          100  261653.611        48.485798     118.49   
           3    7198.7          144  351712.733        48.857812     118.47   
           4    4457.0            0  225938.810        50.693024     177.66   
           5    3750.4           24  179068.882        47.746609     181.40   
...                ...          ...         ...              ...        ...   
2021 5     28   1778.9           96  169945.495        95.534035     298.83   
           29   3695.1          100  146378.094        39.614109      64.96   
           30   3280.0          120  113233.630        34.522448      40.08   
           31   1684.6            0  163582.794        97.104828     298.06   
     6     1    1630.0           24  190001.090       116.565086     299.69   

                price_min  price_mean  power_max  power_min  power_mean  \
year month day                                                            
2016 1     1        35.09   45.982500      502.9       53.0  328.075000   
           2        40.03   56.166500      486.0       40.0  269.825000   
           3        44.06   54.959167      486.0       40.0  299.945833   
           4        46.70   80.164583      486.0        0.0  185.708333   
           5        42.06   84.503333      436.0        0.0  156.266667   
...                   ...         ...        ...        ...         ...   
2021 5     28       36.03   97.538750      194.0       38.0   74.120833   
           29       27.18   41.100500      246.0      106.0  184.755000   
           30       27.18   35.110000      286.0      102.0  164.000000   
           31       36.12  109.443000      198.0       53.8   84.230000   
     6     1        41.18  128.647500      124.0        0.0   67.916667   

                price_morning_max  price_evening_max  price_standard  
year month day                                                        
2016 1     1                48.03              60.08         45.8275  
           2                56.05             118.49         50.2275  
           3                56.01             118.47         50.6050  
           4                80.02             177.66         80.0200  
           5               148.40             181.40         80.0500  
...                           ...                ...             ...  
2021 5     28              167.67             298.83         77.0425  
           29               45.44              64.96         36.2825  
           30               35.56              40.08         35.5175  
           31              167.57             298.06         77.0650  
     6     1               204.16             299.69         80.0825  

[1979 rows x 13 columns]

In [9]:
hourly['cat']='off'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4,19147.750,off
2016-01-01,40.08,486.6,1,2016,1,1,4,19502.928,off
2016-01-01,40.06,495.6,2,2016,1,1,4,19853.736,off
2016-01-01,40.08,496.6,3,2016,1,1,4,19903.728,off
2016-01-01,40.07,502.9,4,2016,1,1,4,20151.203,off
...,...,...,...,...,...,...,...,...,...
2021-06-01,299.34,33.0,19,2021,6,1,1,9878.220,evening
2021-06-01,124.05,60.0,20,2021,6,1,1,7443.000,standard
2021-06-01,80.14,82.0,21,2021,6,1,1,6571.480,standard


In [10]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume  day_of_week       value  price_weight_av  price_max  \
year month day                                                                
2016 1     1    7873.8           96  336882.508        42.785251      60.08   
           2    5396.5          100  261653.611        48.485798     118.49   
           3    7198.7          144  351712.733        48.857812     118.47   
           4    4457.0            0  225938.810        50.693024     177.66   
           5    3750.4           24  179068.882        47.746609     181.40   
...                ...          ...         ...              ...        ...   
2021 5     28   1778.9           96  169945.495        95.534035     298.83   
           29   3695.1          100  146378.094        39.614109      64.96   
           30   3280.0          120  113233.630        34.522448      40.08   
           31   1684.6            0  163582.794        97.104828     298.06   
     6     1    1630.0           24  190001.090       116.565086     299.69   

                price_min  price_mean  power_max  power_min  power_mean  ...  \
year month day                                                           ...   
2016 1     1        35.09   45.982500      502.9       53.0  328.075000  ...   
           2        40.03   56.166500      486.0       40.0  269.825000  ...   
           3        44.06   54.959167      486.0       40.0  299.945833  ...   
           4        46.70   80.164583      486.0        0.0  185.708333  ...   
           5        42.06   84.503333      436.0        0.0  156.266667  ...   
...                   ...         ...        ...        ...         ...  ...   
2021 5     28       36.03   97.538750      194.0       38.0   74.120833  ...   
           29       27.18   41.100500      246.0      106.0  184.755000  ...   
           30       27.18   35.110000      286.0      102.0  164.000000  ...   
           31       36.12  109.443000      198.0       53.8   84.230000  ...   
     6     1        41.18  128.647500      124.0        0.0   67.916667  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2016 1     1                   3.0                 2.0      3851.5   
           2                   NaN                 NaN      4046.5   
           3                   NaN                 NaN      7198.7   
           4                   3.0                 2.0      3828.0   
           5                   3.0                 2.0      3230.0   
...                            ...                 ...         ...   
2021 5     28                  3.0                 2.0       458.0   
           29                  NaN                 NaN      2623.0   
           30                  NaN                 NaN      3280.0   
           31                  3.0                 2.0       402.0   
     6     1                   3.0                 3.0       463.0   

                standard_volume  morning_volume  evening_volume   off_value  \
year month day                                                                
2016 1     1             2920.3           993.0           109.0  152306.418   
           2             1350.0             NaN             NaN  187238.611   
           3                NaN             NaN             NaN  351712.733   
           4              464.0           109.0            56.0  179880.680   
           5              368.4           109.0            43.0  136366.420   
...                         ...             ...             ...         ...   
2021 5     28             979.0           228.0           113.9   17277.250   
           29            1072.1             NaN             NaN   93158.670   
           30               NaN             NaN             NaN  113233.630   
           31             946.0           222.8           113.8   15955.460   
     6     1              838.0           250.0            79.0   22324.050   

        

In [11]:
daily.to_csv(out + pref + '_daily.csv')

In [12]:
hourly.to_csv(out + pref + '_hourly.csv')

In [13]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2016,1,3723.319355,74.064516,209210.018032,106.684000,45.982500,82.524992,362.651613,17.606452,...,42.0,56.189115,50.532675,81.002858,77.850006,137.720998,270.863305,33.409524,56.583459,37.350000
1,2016,2,3831.906897,69.379310,259008.811241,123.798500,75.023333,92.112261,367.334483,23.231034,...,42.0,67.592668,56.664659,106.547628,129.026815,156.801247,274.478704,12.280952,75.799614,42.580952
2,2016,3,2859.093548,68.645161,185401.480839,108.651500,60.115833,89.230347,285.867742,18.232258,...,46.0,64.846245,55.377722,98.717668,116.099471,107.341031,210.890964,15.502899,56.748043,34.786957
3,2016,4,1890.450000,72.533333,117367.291100,114.641667,42.853500,85.768178,183.970000,7.236667,...,42.0,62.084314,57.206680,104.192200,104.410448,139.262082,151.238576,7.793651,19.055263,4.438095
4,2016,5,1591.100000,69.548387,89728.796387,105.676500,28.659583,79.426629,165.183871,4.661290,...,44.0,56.394190,49.773472,106.019186,95.368696,140.331232,124.984770,10.831818,18.262963,4.170455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2021,2,2199.767857,71.857143,182032.625500,136.436250,43.510417,85.344339,141.325000,48.175000,...,40.0,82.750834,56.516119,96.540477,124.672487,160.354642,91.810312,97.925000,96.015726,63.170000
62,2021,3,2718.458065,66.838710,157374.767194,84.241000,27.023333,61.736844,189.854839,58.119355,...,46.0,57.891188,40.687409,61.365324,107.069933,131.375021,120.453614,87.672464,122.960142,80.008696
63,2021,4,2658.506667,71.333333,160156.721933,110.314000,30.873750,80.874203,185.530000,40.316667,...,44.0,60.243115,37.313711,66.760284,142.181426,252.474165,131.762805,74.968182,109.424815,46.463636
64,2021,5,2485.690323,71.354839,167772.532516,113.129583,34.269167,84.048180,162.409677,48.145161,...,42.0,67.495348,37.209169,90.791796,145.617597,285.751219,130.478431,81.076190,87.254511,51.383333


In [14]:
monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2016,1,3723.319355,74.064516,209210.018032,106.684000,45.982500,82.524992,362.651613,17.606452,...,81.002858,77.850006,137.720998,270.863305,33.409524,56.583459,37.350000,21,21,31
1,2016,2,3831.906897,69.379310,259008.811241,123.798500,75.023333,92.112261,367.334483,23.231034,...,106.547628,129.026815,156.801247,274.478704,12.280952,75.799614,42.580952,21,21,29
2,2016,3,2859.093548,68.645161,185401.480839,108.651500,60.115833,89.230347,285.867742,18.232258,...,98.717668,116.099471,107.341031,210.890964,15.502899,56.748043,34.786957,23,23,31
3,2016,4,1890.450000,72.533333,117367.291100,114.641667,42.853500,85.768178,183.970000,7.236667,...,104.192200,104.410448,139.262082,151.238576,7.793651,19.055263,4.438095,21,21,30
4,2016,5,1591.100000,69.548387,89728.796387,105.676500,28.659583,79.426629,165.183871,4.661290,...,106.019186,95.368696,140.331232,124.984770,10.831818,18.262963,4.170455,22,22,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2021,2,2199.767857,71.857143,182032.625500,136.436250,43.510417,85.344339,141.325000,48.175000,...,96.540477,124.672487,160.354642,91.810312,97.925000,96.015726,63.170000,20,20,28
62,2021,3,2718.458065,66.838710,157374.767194,84.241000,27.023333,61.736844,189.854839,58.119355,...,61.365324,107.069933,131.375021,120.453614,87.672464,122.960142,80.008696,23,23,31
63,2021,4,2658.506667,71.333333,160156.721933,110.314000,30.873750,80.874203,185.530000,40.316667,...,66.760284,142.181426,252.474165,131.762805,74.968182,109.424815,46.463636,22,22,30
64,2021,5,2485.690323,71.354839,167772.532516,113.129583,34.269167,84.048180,162.409677,48.145161,...,90.791796,145.617597,285.751219,130.478431,81.076190,87.254511,51.383333,21,21,31


In [15]:
monthly.to_csv(out + pref + '_monthly.csv')

In [16]:
monthly_filt=monthly.loc[monthly['year']<2020]
daily_filt=daily.reset_index().loc[daily.reset_index()['year']<2020]

In [17]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]

calmonthly

,month,day_of_week,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,off_value,standard_value,morning_value,evening_value,volume,off_volume,standard_volume,morning_volume,evening_volume,value
0,1,69.774194,171666.329944,46.371946,97.634677,33.017016,57.820790,302.916129,46.225000,163.251142,...,2.995960e+06,1.756499e+06,3.278078e+05,2.413896e+05,118868.075,82978.700,27816.775,5167.975,2904.625,5.321656e+06
1,2,70.902655,179423.445540,49.513375,103.178584,32.019823,59.784735,290.303540,39.270796,152.720752,...,2.530223e+06,1.898264e+06,3.678901e+05,2.723345e+05,102832.175,68237.550,26354.975,5379.275,2860.375,5.068712e+06
2,3,71.354839,122402.390056,46.570269,98.810161,29.082177,54.417055,206.841935,41.492742,109.351761,...,1.682007e+06,1.472769e+06,3.913917e+05,2.483061e+05,79647.375,46843.400,23659.500,6207.825,2936.650,3.794474e+06
3,4,71.300000,147404.884967,48.025984,97.065000,27.761500,54.422238,211.321667,53.966667,127.339167,...,1.645878e+06,1.793427e+06,5.844292e+05,3.984127e+05,90244.375,48292.600,29595.050,8184.400,4172.325,4.422147e+06
4,5,68.870968,122323.816000,47.346906,111.561774,29.676774,58.582787,206.340323,36.632258,104.511492,...,1.744271e+06,1.350093e+06,4.391428e+05,2.585312e+05,76102.975,47780.775,20351.700,5538.400,2432.100,3.792038e+06
5,6,71.933333,214711.881658,60.080347,135.855750,37.025750,73.826056,293.115833,43.570000,146.398917,...,2.854530e+06,2.029336e+06,8.091018e+05,7.483890e+05,103764.400,68085.200,23696.300,6554.725,5428.175,6.441356e+06
6,7,70.451613,287816.407839,65.865438,145.048952,40.207581,79.184351,343.320968,56.512903,181.216499,...,3.687076e+06,2.976411e+06,1.215369e+06,1.043452e+06,131627.750,82985.400,32715.200,8958.050,6969.100,8.922309e+06
7,8,69.548387,317472.327968,63.753444,146.385161,37.332581,78.534230,403.985484,67.423387,213.005081,...,4.053262e+06,3.413310e+06,1.255548e+06,1.119523e+06,155456.425,97071.375,40083.425,9914.150,8387.475,9.841642e+06
8,9,72.200000,327097.549725,53.169148,119.131000,33.549167,64.407161,462.526667,107.923333,261.611194,...,4.399405e+06,3.696803e+06,1.060691e+06,6.560265e+05,185693.425,116992.075,51069.725,11650.225,5981.400,9.812926e+06
9,10,69.096774,350956.503573,49.894555,105.153306,31.923226,57.045784,494.074194,116.862097,305.219704,...,4.528541e+06,4.676012e+06,1.138204e+06,5.368942e+05,222583.150,134873.175,69330.775,13605.925,4773.275,1.087965e+07


In [18]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [19]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price      volume         value
month hour                                      
1     0      38.854444  234.450617   8978.919562
      1      38.095802  235.047531   8928.320704
      2      37.743333  234.820988   8884.272938
      3      37.639136  237.233951   8975.272481
      4      37.673172  238.518280   9002.655156
...                ...         ...           ...
12    19    130.661613   80.230968   8684.619781
      20     89.013548  105.837419   7205.543774
      21     73.281871  171.339355  10649.464465
      22     39.710839  254.710968   8996.688987
      23     37.374000  260.658710   9130.851213

[288 rows x 3 columns]

In [20]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')